In [1]:
import json, pandas as pd, numpy as np
from datetime import datetime
from conceptual_engineering_toolkit import Concept, Entity
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WikipediaLoader

In [2]:
SAMPLE_SIZE = 4
females = np.random.choice(json.load(open('data/females-2023-03-15.json', 'r')), SAMPLE_SIZE)
males = np.random.choice(json.load(open('data/males-2023-03-15.json', 'r')), SAMPLE_SIZE)
non_binaries = np.random.choice(json.load(open('data/non-binaries-2023-03-15.json', 'r')), SAMPLE_SIZE)
trans_men = np.random.choice(json.load(open('data/trans-men-2023-03-15.json', 'r')), SAMPLE_SIZE)
trans_women = np.random.choice(json.load(open('data/trans-women-2023-03-15.json', 'r')), SAMPLE_SIZE)
sample = np.concatenate((females, trans_women, non_binaries, trans_men, males))

In [3]:
for entity in sample:
    entity["description"] = WikipediaLoader(query=entity["name"], load_max_docs=1).load()[0].page_content[:1200]

In [4]:
entities = pd.DataFrame.from_records(sample)
woman_conditions = [
    ((entities.gender == 'female') | (entities.gender == 'trans woman')),
    ((entities.gender == 'non-binary') | (entities.gender == 'male') | (entities.gender == 'trans man'))
]
woman_categories = ['true', 'false']
entities['woman'] = np.select(woman_conditions, woman_categories)
entities

,item,name,article,gender,description,woman
0,http://www.wikidata.org/entity/Q11481,Kathleen Booth,https://en.wikipedia.org/wiki/Kathleen_Booth,female,"Kathleen Hylda Valerie Booth (née Britten, 9 J...",true
1,http://www.wikidata.org/entity/Q65719,Anke Wischnewski,https://en.wikipedia.org/wiki/Anke_Wischnewski,female,Anke Wischnewski (born 5 January 1978 in Annab...,true
2,http://www.wikidata.org/entity/Q69173,Dorothea Röschmann,https://en.wikipedia.org/wiki/Dorothea_R%C3%B6...,female,Dorothea Röschmann (born 17 June 1967) is a Ge...,true
3,http://www.wikidata.org/entity/Q92267,Princess Casimire of Anhalt-Dessau,https://en.wikipedia.org/wiki/Princess_Casimir...,female,"Casimire of Anhalt-Dessau (19 January 1749, De...",true
4,http://www.wikidata.org/entity/Q2635912,Jennifer Ventimilia,https://en.wikipedia.org/wiki/Jennifer_Ventimilia,trans woman,Jennifer Ventimilia (previously Jeffrey Ventim...,true
5,http://www.wikidata.org/entity/Q2245540,Eva Robin's,https://en.wikipedia.org/wiki/Eva_Robin%27s,trans woman,Eva Robin's (10 December 1958) is a transgende...,true
6,http://www.wikidata.org/entity/Q96474820,Backxwash,https://en.wikipedia.org/wiki/Backxwash,trans woman,"Ashanti Mutinta, known professionally as Backx...",true
7,http://www.wikidata.org/entity/Q5551765,Natalia Parés Vives,https://en.wikipedia.org/wiki/Natalia_Par%C3%A...,trans woman,Natalia Parés Vives (born 1955) is a chess pla...,true
8,http://www.wikidata.org/entity/Q90013819,Maxi Glamour,https://en.wikipedia.org/wiki/Maxi_Glamour,non-binary,Maxi Glamour is a non-binary multi-disciplinar...,false
9,http://www.wikidata.org/entity/Q19561491,Catherine McKenney,https://en.wikipedia.org/wiki/Catherine_McKenney,non-binary,"Catherine McKenney (born June 3, 1961) is a Ca...",false


In [5]:
oed_woman = Concept("oed_woman", "woman", "An adult female human being. The counterpart of man (see man, n.¹ II.4.)")

In [6]:
haslanger_woman = Concept("haslanger_woman", "woman", """S is a woman iff (i) S is regularly and for the most part observed or imagined to have 
certain bodily features presumed to be evidence of a female’s biological role in reproduction; 
(ii) that S has these features marks S within the dominant ideology of S’s society as someone who 
ought to occupy certain kinds of social position that are in fact subordinate (and so motivates 
and justifies S’s occupying such a position); and (iii) the fact that S satisfies (I) and (ii) plays 
a role in S’s systematic subordination, that is, along some dimension, S’s social position is oppressive, 
and S’s satisfying (i) and (ii) plays a role in that dimension of subordination""")

In [7]:
homosaurus_woman = Concept("homosaurus_woman", "woman", """Adults who self-identify as women and understand their gender in terms of Western conceptions of 
womanness, femaleness, and/or femininity. The term has typically been defined as adult female humans, though not all women identify with 
the term "female" depending on the context in which it is used.""")

In [8]:
oed_classifications = pd.DataFrame( 
    [ oed_woman.classify(Entity(entity["item"], entity["name"], entity["description"])) for entity in sample ], 
    columns = [ 'entity' , 'classification', 'rationale' ] 
    )
oed_classifications["classification"] = oed_classifications["classification"].str.lower()

In [9]:
oed_results = pd.concat([entities[["name", "gender", "woman"]], oed_classifications[["classification", "rationale"]]], axis=1)
results_styler = oed_results.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})
results_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) 

,name,gender,woman,classification,rationale
0,Kathleen Booth,female,true,true,"The text identifies Kathleen Booth as an adult human being who is referred to using female pronouns such as ""she"" and ""her"". Additionally, she is described as having been born with the female name Kathleen Britten, and later getting married and having children, which are traditionally female roles. Therefore, based on the provided definition of a woman as an adult female human being, it can be argued that Kathleen Booth is a woman."
1,Anke Wischnewski,female,true,true,"The description of Anke Wischnewski provides information that she is an adult human being who has competed in luge events. The description also refers to her using female pronouns such as ""she"". Furthermore, she has won medals in the women's singles at the FIL World Luge Championships and competed in the women's singles event at the FIL European Luge Championships. This information aligns with the definition of a woman as an adult female human being. Therefore, it can be argued that Anke Wischnewski is a woman."
2,Dorothea Röschmann,female,true,true,"The definition of a woman is an adult female human being. The description of Dorothea Röschmann indicates that she is an adult human being, as she was born in 1967 and has had a career in music. Furthermore, the use of female pronouns such as ""she"" and ""her"" in the description of Röschmann indicates that she is female. Therefore, based on the definition and the information provided, Dorothea Röschmann can be classified as a woman."
3,Princess Casimire of Anhalt-Dessau,female,true,true,"Princess Casimire of Anhalt-Dessau is a woman because the description provides information that she is an adult female human being. This is evident from the use of female pronouns such as ""she"" and ""her"" throughout the description. Additionally, she is referred to as a ""princess"" and a ""countess"", both of which are titles typically given to adult females. Furthermore, she was married, which traditionally implies adulthood. Therefore, based on the definition of a woman as an adult female human being, Princess Casimire of Anhalt-Dessau can be classified as a woman."
4,Jennifer Ventimilia,trans woman,true,true,"The text refers to Jennifer Ventimilia using female pronouns such as ""she"" and ""her"". This indicates that Jennifer Ventimilia is identified as a female. Since she is also described as an adult, being a professional screenwriter and producer, it can be inferred that Jennifer Ventimilia is a woman, as per the definition provided."
5,Eva Robin's,trans woman,true,true,"Eva Robin's can be considered a woman based on the provided definition of a woman as an adult female human being. The description states that Eva identifies as female and has been living as a woman since the age of twenty-one. She has also been introduced to feminizing hormones, further aligning her identity with that of a woman. Therefore, based on her self-identification and lifestyle, Eva Robin's fits the definition of a woman."
6,Backxwash,trans woman,true,true,"The text states that Ashanti Mutinta, known professionally as Backxwash, came out as transgender. Given the definition of a woman as an adult female human being, and the fact that Backxwash identifies as a female, it can be argued that Backxwash is a woman."
7,Natalia Parés Vives,trans woman,true,true,"The text describes Natalia Parés Vives as an ""openly trans woman"". The term ""woman"" in this context refers to her gender identity as a female. Therefore, based on the definition provided, which states that a woman is an adult female human being, it can be argued that Natalia Parés Vives is a woman."
8,Maxi Glamour,non-binary,false,false,"Based on the provided definition and information, there is no evidence to support the assertion that Maxi Glamour is a woman. The description clearly states that Maxi Glamour is a non-binary individual, which means they do not identify as exclusively male or female. Therefo

In [10]:
haslanger_classifications = pd.DataFrame( 
    [ haslanger_woman.classify(Entity(entity["item"], entity["name"], entity["description"])) for entity in sample ], 
    columns = [ 'entity' , 'classification', 'rationale' ] 
    )
haslanger_classifications["classification"] = haslanger_classifications["classification"].str.lower()

In [11]:
haslanger_results = pd.concat([entities[["name", "gender", "woman"]], haslanger_classifications[["classification", "rationale"]]], axis=1)
results_styler = haslanger_results.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})
results_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) 

,name,gender,woman,classification,rationale
0,Kathleen Booth,female,true,true,"Kathleen Booth can be considered a woman based on the provided definition. Firstly, she is referred to with female pronouns such as ""she"" and ""her"", which suggests that she is regularly and for the most part observed or imagined to have certain bodily features presumed to be evidence of a female’s biological role in reproduction (i). Secondly, the fact that she was a computer scientist and mathematician in a time when these fields were dominated by men could suggest that she was marked within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate (ii). Lastly, the fact that she was a research assistant to her husband could be seen as a role in her systematic subordination, as her social position could be considered oppressive in the context of her time (iii). Therefore, based on these points, Kathleen Booth can be classified as a woman."
1,Anke Wischnewski,female,true,true,"Anke Wischnewski can be considered a woman based on the provided definition. Firstly, she is observed to have certain bodily features presumed to be evidence of a female’s biological role in reproduction. This is inferred from the fact that she competed in women's singles events in luge, which are typically segregated by sex. Secondly, her participation in these events marks her within the dominant ideology of her society as someone who ought to occupy certain kinds of social position. This is evident from her participation in a sport that is traditionally gender-segregated, indicating societal expectations about the roles and activities appropriate for her sex. Lastly, the fact that she satisfies the first two conditions plays a role in her systematic subordination. While the text does not explicitly state that she is oppressed, it can be inferred that as a woman in a competitive sport, she likely faces challenges and obstacles related to her gender. Therefore, based on the provided definition and the information about Anke Wischnewski, it can be argued that she is a woman."
2,Dorothea Röschmann,female,true,true,"From the provided description, we can infer that Dorothea Röschmann is a woman based on the following points: (i) The description refers to Dorothea Röschmann using female pronouns (""she"", ""her""), which suggests that she is observed or imagined to have certain bodily features presumed to be evidence of a female’s biological role in reproduction. (ii) Dorothea Röschmann is a soprano, a term typically used to describe the highest singing voice in women. This suggests that she is marked within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are in fact subordinate. (iii) The fact that Dorothea Röschmann is a soprano and is referred to with female pronouns plays a role in her systematic subordination. In the world of opera, sopranos often play roles that are subordinate to male roles, which could be seen as oppressive. This satisfies the third condition of the definition. Therefore, based on the provided definition and the information given about Dorothea Röschmann, we can argue that Dorothea Röschmann is a woman."
3,Princess Casimire of Anhalt-Dessau,female,true,true,"Princess Casimire of Anhalt-Dessau can be considered a woman based on the provided definition. Firstly, she is presumed to have certain bodily features associated with a female's biological role in reproduction, as indicated by her title of 'Princess' and her marriage to Count Simon August of Lippe-Detmold. Secondly, her societal role as a princess and countess marks her within the dominant ideology of her society as someone who ought to occupy certain kinds of social position that are subordinate, as was typical for women of her time. Lastly, her societal roles and her biological features contribute to her systematic subordination, as she was expected to fulfill certain

In [12]:
homosaurus_classifications = pd.DataFrame( 
    [ homosaurus_woman.classify(Entity(entity["item"], entity["name"], entity["description"])) for entity in sample ], 
    columns = [ 'entity' , 'classification', 'rationale' ] 
    )
homosaurus_classifications["classification"] = homosaurus_classifications["classification"].str.lower()

In [13]:
homosaurus_results = pd.concat([entities[["name", "gender", "woman"]], homosaurus_classifications[["classification", "rationale"]]], axis=1)
results_styler = homosaurus_results.style.set_properties(**{"text-align": "left", "overflow-wrap": "break-word"})
results_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]) 

,name,gender,woman,classification,rationale
0,Kathleen Booth,female,true,true,"Kathleen Booth can be identified as a woman based on the provided definition. The definition states that a woman is an adult who self-identifies as a woman and understands their gender in terms of Western conceptions of womanness, femaleness, and/or femininity. Kathleen Booth is an adult, as indicated by her birth date and the timeline of her life events. She is referred to using female pronouns (""she"", ""her""), which suggests that she identifies as a woman. Furthermore, there is no information suggesting that she does not understand her gender in terms of Western conceptions of womanness, femaleness, and/or femininity. Therefore, based on the provided definition and the information about Kathleen Booth, it can be argued that Kathleen Booth is a woman."
1,Anke Wischnewski,female,true,true,"Anke Wischnewski is identified as a woman based on the information provided in the description. She has competed in women's singles events in luge championships, which implies that she identifies as a woman. Furthermore, the use of female pronouns to describe her also supports the assertion that she identifies as a woman. Therefore, according to the definition provided, Anke Wischnewski can be classified as a woman."
2,Dorothea Röschmann,female,true,true,"The text identifies Dorothea Röschmann as a ""she"", which is a pronoun typically used in English to refer to individuals who self-identify as women. Furthermore, the text does not provide any information that would suggest Röschmann identifies her gender in any way that contradicts Western conceptions of womanness, femaleness, and/or femininity. Therefore, based on the provided definition and the information given about Röschmann, it can be argued that Dorothea Röschmann is a woman."
3,Princess Casimire of Anhalt-Dessau,female,true,true,"Princess Casimire of Anhalt-Dessau can be identified as a woman based on the provided definition. She is an adult who presumably self-identifies as a woman, given her title of ""Princess"" and the use of female pronouns in the description. She also understands her gender in terms of Western conceptions of womanness, femaleness, and/or femininity, as evidenced by her role as a princess and countess, positions traditionally held by women in Western societies. Furthermore, she was married to a man, Count Simon August of Lippe-Detmold, which aligns with traditional Western gender roles. Therefore, based on the provided definition, Princess Casimire of Anhalt-Dessau can be classified as a woman."
4,Jennifer Ventimilia,trans woman,true,true,"The text states that Jennifer Ventimilia was previously known as Jeffrey Ventimilia, which suggests a change in gender identity. Given that she now identifies as Jennifer, it can be inferred that she self-identifies as a woman. This aligns with the definition of a woman as an adult who self-identifies as a woman and understands their gender in terms of Western conceptions of womanness, femaleness, and/or femininity. Therefore, based on the provided definition and the information about Jennifer Ventimilia, it can be argued that Jennifer Ventimilia is a woman."
5,Eva Robin's,trans woman,true,true,"Eva Robin's can be considered a woman based on the provided definition. The definition states that a woman is an adult who self-identifies as a woman and understands their gender in terms of Western conceptions of womanness, femaleness, and/or femininity. Eva Robin's, from the age of thirteen, felt she was female and by the age of twenty-one, she was living as a woman. This indicates that she self-identifies as a woman and understands her gender in terms of womanness and femininity. The definition also acknowledges that not all women identify with the term ""female"" depending on the context, which aligns with Eva's comfort with her body and lack of desire for sex reassignment surgery. Therefore, based on the provided definition, Eva Robin's can be cons

In [14]:
def evaluation_record(concept, data):
    return {
        "created": datetime.now().isoformat(),
        "concept": concept.to_json(),
        "data": data.to_dict('records'),
    }

In [15]:
oed_evaluation_record = evaluation_record(oed_woman, pd.concat([entities, oed_classifications], axis=1))
haslanger_evaluation_record = evaluation_record(haslanger_woman, pd.concat([entities, haslanger_classifications], axis=1))
homosaurus_evaluation_record = evaluation_record(homosaurus_woman, pd.concat([entities, homosaurus_classifications], axis=1))
json.dump(
    oed_evaluation_record, 
    open(f'data/{oed_evaluation_record["concept"]["model_name"]}_{oed_evaluation_record["concept"]["id"]}_{oed_evaluation_record["created"]}.json', 'w')
)
json.dump(
    haslanger_evaluation_record, 
    open(f'data/{haslanger_evaluation_record["concept"]["model_name"]}_{haslanger_evaluation_record["concept"]["id"]}_{haslanger_evaluation_record["created"]}.json', 'w')
)
json.dump(
    homosaurus_evaluation_record, 
    open(f'data/{homosaurus_evaluation_record["concept"]["model_name"]}_{homosaurus_evaluation_record["concept"]["id"]}_{homosaurus_evaluation_record["created"]}.json', 'w')
)